# Part 1. Load the movie ratings data and predict the missing ratings from the test data by using sklearn's non-negative matrix facorization library.

## - load the movie rating data (as in the HW3-recommender-system) 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix, csr_matrix
from scipy.spatial.distance import jaccard, cosine 
from pytest import approx

from sklearn.decomposition import NMF
import warnings
from numpy import zeros

In [2]:
# Read the ratings data
ratingData = pd.read_csv('ratings.dat', names=['uID', 'mID', 'rating', 'time'], engine='python', delimiter='::')
ratingData.head()

,uID,mID,rating,time
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
ratingData['rating'].describe()

count    1.000209e+06
mean     3.581564e+00
std      1.117102e+00
min      1.000000e+00
25%      3.000000e+00
50%      4.000000e+00
75%      4.000000e+00
max      5.000000e+00
Name: rating, dtype: float64

In [4]:
train, test = train_test_split(ratingData, test_size=0.2, random_state=42)

In [5]:
# Read the movie data
MV_movies = pd.read_csv('movies.dat', names=['mID', 'title', 'genre'], engine='python', delimiter='::',encoding='latin-1')
MV_movies.head()

,mID,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
column_names = ['uID', 'Gender', 'Age', 'Occupation', 'Zip-code']
MV_users = pd.read_csv("users.dat", sep = "::", names = column_names, engine='python')
MV_users.head()
#users.drop('Zip-code',axis = 1).head()

,uID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [7]:
from collections import namedtuple
Data = namedtuple('Data', ['users','movies','train','test'])
data = Data(MV_users, MV_movies, train, test)

In [8]:
class RecSys():
    def __init__(self,data):
        self.data=data
        self.allusers = list(self.data.users['uID'])
        self.allmovies = list(self.data.movies['mID'])
        self.genres = list(self.data.movies.columns.drop(['mID', 'title']))
        self.mid2idx = dict(zip(self.data.movies.mID,list(range(len(self.data.movies)))))
        self.uid2idx = dict(zip(self.data.users.uID,list(range(len(self.data.users)))))
        self.Mr=self.rating_matrix()
        self.Mm=None 
        self.sim=np.zeros((len(self.allmovies),len(self.allmovies)))
        
    def rating_matrix(self):
        """
        Convert the rating matrix to numpy array of shape (#allusers,#allmovies)
        """
        ind_movie = [self.mid2idx[x] for x in self.data.train.mID] 
        ind_user = [self.uid2idx[x] for x in self.data.train.uID]
        rating_train = list(self.data.train.rating)
        
        return np.array(coo_matrix((rating_train, (ind_user, ind_movie)), shape=(len(self.allusers), len(self.allmovies))).toarray())


    def predict_everything_to_3(self):
        """
        Predict everything to 3 for the test data
        """
        y_3=np.array(self.data.test.rating)
        y_3[:]=3
        return y_3 
    
        
    def predict_to_user_average(self):
        """
        Predict to average rating for the user.
        Returns numpy array of shape (#users,)
        """
        ave = self.data.train.groupby(['uID']).mean() 
        ave = ave.drop('mID',axis=1)
        ave = ave.rename(columns={'rating': 'rating_ave'})
        y_ave = pd.merge(self.data.test, ave, on="uID", how="left")
        return y_ave.rating_ave 
    
    
    def rmse(self,yp):
        yp[np.isnan(yp)]=3 #In case there is nan values in prediction, it will impute to 3.
        yt=np.array(self.data.test.rating)
        return np.sqrt(((yt-yp)**2).mean())

In [9]:
# Creating Sample test data
np.random.seed(42)
sample_train = train[:500]
sample_test = test[:500]

sample_MV_users = MV_users[(MV_users.uID.isin(sample_train.uID)) | (MV_users.uID.isin(sample_test.uID))]
sample_MV_movies = MV_movies[(MV_movies.mID.isin(sample_train.mID)) | (MV_movies.mID.isin(sample_test.mID))]

sample_data = Data(sample_MV_users, sample_MV_movies, sample_train, sample_test)

In [10]:
# Sample tests predict_everything_to_3 in class RecSys
sample_rs = RecSys(sample_data)
sample_yp = sample_rs.predict_everything_to_3()
print(sample_rs.rmse(sample_yp))

1.2449899597988732


In [11]:
# Sample tests predict_to_user_average in the class RecSys
sample_yp = sample_rs.predict_to_user_average()
print(sample_rs.rmse(sample_yp))

1.300769003320728


In [12]:
warnings.simplefilter('ignore')
sample_test['ind_user'] = sample_test['uID'].map(sample_rs.uid2idx)
sample_test['ind_movie'] = sample_test['mID'].map(sample_rs.mid2idx)

## - predict the missing ratings from the test data by using sklearn's non-negative matrix facorization library.

In [13]:
#NMF

warnings.simplefilter('ignore')

for k in range(10,21):
    model = NMF(n_components=k, init='random', max_iter=100, random_state=0)
    P = model.fit_transform(sample_rs.Mr)
    Q = model.components_
    
    def predict(uID, mID):
        return np.dot(P[uID], Q[:, mID])
    
    sample_test['predict'] = sample_test[['ind_user', 'ind_movie']].apply(lambda text: predict(text[0], text[1]), axis=1)
    print("n_components =",k, " ->  RMSE =",  sample_rs.rmse(sample_test.predict))

n_components = 10  ->  RMSE =  3.7312196397424797
n_components = 11  ->  RMSE =  3.7312196397424797
n_components = 12  ->  RMSE =  3.7312196397424797
n_components = 13  ->  RMSE =  3.7312196397424797
n_components = 14  ->  RMSE =  3.7312196397424797
n_components = 15  ->  RMSE =  3.7312196397424797
n_components = 16  ->  RMSE =  3.7312196397424797
n_components = 17  ->  RMSE =  3.7312196397424797
n_components = 18  ->  RMSE =  3.7312196397424797
n_components = 19  ->  RMSE =  3.7312196397424797
n_components = 20  ->  RMSE =  3.7312196397424797


In [14]:
sample_test.head()

,uID,mID,rating,time,ind_user,ind_movie,predict
895536,5412,2683,2,960243649,747,503,0.0
899739,5440,904,5,959995181,751,155,0.0
55687,368,3717,4,976311423,43,683,0.0
63727,425,1721,4,976283587,51,332,0.0
822011,4942,3697,1,962642480,679,670,0.0


In [15]:
sample_test.describe()

,uID,mID,rating,time,ind_user,ind_movie,predict
count,500.000000,500.000000,500.000000,5.000000e+02,500.000000,500.00000,5.000000e+02
mean,3087.670000,1843.576000,3.562000,9.716229e+08,423.378000,345.83600,2.477180e-52
std,1781.291636,1134.030311,1.112038,1.178799e+07,247.791542,210.03778,5.522062e-51
min,10.000000,9.000000,1.000000,9.567165e+08,0.000000,2.00000,0.000000e+00
25%,1416.000000,917.500000,3.000000,9.651904e+08,205.750000,161.50000,0.000000e+00
50%,3262.000000,1720.000000,4.000000,9.712517e+08,427.000000,331.50000,0.000000e+00
75%,4549.250000,2764.750000,4.000000,9.751198e+08,629.250000,526.25000,0.000000e+00
max,6040.000000,3925.000000,5.000000,1.045456e+09,839.000000,714.00000,1.234772e-49


-> sklearn's non-negative matrix facorization library did not work well compared to simple baseline we’ve done in Module 3.<br>
-> Predict values are very small and RMSE=3.73.

One reason is that sklearn's non-negative matrix facorization library considers all elements, including the "0" element, when updating the optimization. This can be fatal when dealing with sparse matrices. A sample of a small matrix is shown below.

In [22]:
R = np.array([
        [0, 0, 3, 0],
        [1, 2, 0, 0],
        [0, 0, 0, 5],
        [1, 0, 0, 3],
        [0, 0, 0, 4],
        ])

model = NMF(n_components=5, init='random', max_iter=200, random_state=0)
P = model.fit_transform(R)
Q = model.components_
print(np.round(np.dot(P, Q), 0))

[[0. 0. 3. 0.]
 [1. 2. 0. 0.]
 [0. 0. 0. 5.]
 [1. 0. 0. 3.]
 [0. 0. 0. 4.]]


-> It has been reproduced including 0.

# Part 2. suggest a way to fix

One possible improvement would be the process of excluding "0" elements when updating the optimization. The actual code and results are shown below

In [17]:
def get_rating_error(r, p, q):
    return r - np.dot(p, q)


def get_error(R, P, Q, beta):
    error = 0.0
    for i in range(len(R)):
        for j in range(len(R[i])):
            if R[i][j] == 0:
                continue
            error += pow(get_rating_error(R[i][j], P[:,i], Q[:,j]), 2)
    error += beta/2.0 * (np.linalg.norm(P) + np.linalg.norm(Q))
    return error


def matrix_factorization(R, K, steps=200, alpha=0.0002, beta=0.001, threshold=0.001):
    P = np.random.rand(K, len(R))
    Q = np.random.rand(K, len(R[0]))
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] == 0:
                    continue
                err = get_rating_error(R[i][j], P[:, i], Q[:, j])
                for k in range(K):
                    P[k][i] += alpha * (2 * err * Q[k][j])
                    Q[k][j] += alpha * (2 * err * P[k][i])
        error = get_error(R, P, Q, beta)
        if error < threshold:
            break
    return P.T, Q

In [18]:
R = np.array([
        [0, 0, 3, 0],
        [1, 2, 0, 0],
        [0, 0, 0, 5],
        [1, 0, 0, 3],
        [0, 0, 0, 4],
        ])

P, Q = matrix_factorization(R, K=5)
print(np.round(np.dot(P, Q), 0))

[[1. 1. 1. 1.]
 [2. 1. 2. 2.]
 [2. 2. 2. 3.]
 [1. 1. 1. 2.]
 [1. 1. 1. 1.]]


-> Elements that were originally 0 also have a value.

In [19]:
#NMF 2 

warnings.simplefilter('ignore')

for k in range(10,21):
    P, Q = matrix_factorization(sample_rs.Mr, K=k)
    
    def predict(uID, mID):
        return min(5, np.dot(P[uID], Q[:, mID]))
    
    sample_test['predict2'] = sample_test[['ind_user', 'ind_movie']].apply(lambda text: predict(text[0], text[1]), axis=1)
    print("n_components =",k, " -> RMSE =",  sample_rs.rmse(sample_test.predict2))

n_components = 10  -> RMSE =  1.6227037698808522
n_components = 11  -> RMSE =  1.4905961950674378
n_components = 12  -> RMSE =  1.3993820182912915
n_components = 13  -> RMSE =  1.3976815733056576
n_components = 14  -> RMSE =  1.3083581635279784
n_components = 15  -> RMSE =  1.3715061792308623
n_components = 16  -> RMSE =  1.4086018551673194
n_components = 17  -> RMSE =  1.3936116567106112
n_components = 18  -> RMSE =  1.4201853548970003
n_components = 19  -> RMSE =  1.532221730241253
n_components = 20  -> RMSE =  1.5984162485871336


-> RMSE improved to 1.308, the result does not significantly different from other methods.